## Tensor概述
对于Tensor的操作很多，从接口角度来讲，可以分为两类
- torch.function，如torch.sum、torch.add等
- tensor.function，如tensor.view、tensor.add等

其中，关于Tensor很多操作都是等价的，如torch.add(x, y)和x.add(y)等价。

从修改方式的角度来看，可以分为两类：
- 不修改自身数据，如x.add(y)，x的数据不变，返回一个新的Tensor。
- 修改自身数据，如x.add_(y)（运算符带下划线），运算结果存在x中。

In [2]:
import torch

In [8]:
x = torch.tensor([1,2])
y = torch.tensor([3,4])
z = x.add(y)
print(x, y, z)
x.add_(z)
print(x)

tensor([1, 2]) tensor([3, 4]) tensor([4, 6])
tensor([5, 8])


### 创建Tensor

In [9]:
# 依据list数据生成Tensor
print(torch.Tensor([1,2,3,4,5,6]))
# 指定形状生成Tensor
print(torch.Tensor(2,3))
# 直接给定Tensor的形状
t = torch.Tensor([[1,2,3],[4,5,6]])
# 查看Tensor形状
print(t.size())
# shape与size()等价方式
print(t.shape)
# 依据已有形状创建Tensor
print(torch.Tensor(t.size()))

tensor([1., 2., 3., 4., 5., 6.])
tensor([[ 3.0829e-44,  0.0000e+00, -5.2688e+26],
        [ 5.1708e-43,  0.0000e+00,  0.0000e+00]])
torch.Size([2, 3])
torch.Size([2, 3])
tensor([[0.0000e+00, 0.0000e+00, 5.4212e-05],
        [2.6080e-09, 1.3553e-05, 2.6807e-09]])


**torch.Tensor**和**torch.tensor**的区别
- torch.Tensor是torch.empty和torch.tensor之间的一种混合，传入数据时，torch.Tensor使用全局默认dtype(FloatTensor)，而torch.tensor是从数据中推断数据类型。
- torch.tensor(1)返回一个固定值1，而torch.Tensor(1)返回的是一大小为1的张量，它是随机初始化的值。

In [10]:
t1 = torch.Tensor(1)
t2 = torch.tensor(1)
print(t1, t1.type(), t2, t2.type())

tensor([inf]) torch.FloatTensor tensor(1) torch.LongTensor


自动生成Tensor的一些例子

In [11]:
# 生成一个单位矩阵
print(torch.eye(2,2))
# 自动生成全是0的矩阵
print(torch.zeros(2,3))
# 
print(torch.linspace(1, 10, 4))
# 生成满足均匀分布随机数
print(torch.rand(2,3))
# 生成满足标准分布随机数
print(torch.randn(2,3))
# 返回所给数据形状相同，值全为0的张量
print(torch.zeros_like(torch.rand(2,3)))

tensor([[1., 0.],
        [0., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([ 1.,  4.,  7., 10.])
tensor([[0.5309, 0.5507, 0.4731],
        [0.9114, 0.0011, 0.4336]])
tensor([[ 0.0360,  0.5883,  0.0856],
        [-0.4142, -1.0687,  1.4036]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


### 修改Tensor形状

In [3]:
x = torch.randn(2, 3)
print(x.size())
print(x.dim())
# 修改x的形状
print(x.view(3,2))
y = x.view(-1)
print(y)
print(y.shape, x.shape)
z = torch.unsqueeze(y, 0)  # 对y添加一个维度
print(z, z.size())
print(z.numel())  #计算z中元素的个数

torch.Size([2, 3])
2
tensor([[-1.5453,  0.5784],
        [-0.9881,  0.1217],
        [ 0.0695,  0.1033]])
tensor([-1.5453,  0.5784, -0.9881,  0.1217,  0.0695,  0.1033])
torch.Size([6]) torch.Size([2, 3])
tensor([[-1.5453,  0.5784, -0.9881,  0.1217,  0.0695,  0.1033]]) torch.Size([1, 6])
6


### 索引操作

In [13]:
# 设置一个随机种子
torch.manual_seed(2020)
# 生成一个2x3的矩阵
x = torch.randn(2, 3)
print(x)
print('The First Row', x[0,:])
print('The last col', x[:, -1])
mask = x > 0
print('生成是否大于0的Byter张量', mask)
# 获取其值
print(torch.masked_select(x, mask))
# 获取非0下标，即行列坐标
print(torch.nonzero(mask))  # False:0, True:1
index = torch.LongTensor([[0, 1, 1]]) 
print('index', index)
# out[i][j][k] = input[index[i][j][k]][j][k]  # dim=0
# out[i][j][k] = input[i][index[i][j][k]][k]  # dim=1
# out[i][j][k] = input[i][j][index[i][j][k]]  # dim=2

print(torch.gather(x, 0, index)) # torch.gather(input, dim, index, out=None) → Tensor
index = torch.LongTensor([[0,1,1], [1,1,1]])
print('index', index)
a = torch.gather(x, 1, index)
print('a', a)
z = torch.zeros(2,3)
print(z)
z.scatter_(1, index, a)
print(z)

tensor([[ 1.2372, -0.9604,  1.5415],
        [-0.4079,  0.8806,  0.0529]])
The First Row tensor([ 1.2372, -0.9604,  1.5415])
The last col tensor([1.5415, 0.0529])
生成是否大于0的Byter张量 tensor([[ True, False,  True],
        [False,  True,  True]])
tensor([1.2372, 1.5415, 0.8806, 0.0529])
tensor([[0, 0],
        [0, 2],
        [1, 1],
        [1, 2]])
index tensor([[0, 1, 1]])
tensor([[1.2372, 0.8806, 0.0529]])
index tensor([[0, 1, 1],
        [1, 1, 1]])
a tensor([[ 1.2372, -0.9604, -0.9604],
        [ 0.8806,  0.8806,  0.8806]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[ 1.2372, -0.9604,  0.0000],
        [ 0.0000,  0.8806,  0.0000]])


### 广播机制

In [14]:
import numpy as np

**numpy广播机制**
- 让所有输入数组都向其中shape最长的数组看齐，不足的部分则通过在前面加1补齐
- 输入数组的shape是输入数组shape的各个轴上的最大值
- 若输入数组的某个轴和输入数组的对应轴的长度想相同或者某个轴的长度为1时，这个数组能被用来计算，否则出错
- 当输入数组的某个轴的长度为1时，沿着此轴运算时都用（或者复制）此轴上的第一组值

In [15]:
# 计算A+B，其中A是4x1矩阵，B是（3，）
A = np.arange(0, 40, 10).reshape(4, 1)
B = np.arange(0, 3)
print(A.shape, B.shape)
print(A, B)
C = A + B
print(C.shape, C)

(4, 1) (3,)
[[ 0]
 [10]
 [20]
 [30]] [0 1 2]
(4, 3) [[ 0  1  2]
 [10 11 12]
 [20 21 22]
 [30 31 32]]


**解释**

由于A的shape长于B的shape，此时B向A看齐，把B变为$(1,3)$, 然后输出值为各轴上的最大值，则输出结果为$(4,3)$。

下面的问题就是$(4,3) \gets (4,1)$以及$(4,3) \gets (1,3)$，则A变为$[[0, 0, 0], [10, 10, 10], [20, 20, 20], [30, 30, 30]]$, $B=[[0, 1, 2]]$变为$[[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1,2]]$

In [20]:

print(A, B)
# ndarray转换为Tensor
A1 = torch.from_numpy(A)
B1 = torch.from_numpy(B)
print(A1, B1)
# Tensor自动实现广播
C = A1 + B1
print(C)
# 手动配置
B2 = B1.unsqueeze(0) # 0前，1后
print(B2)
A2 = A1.expand(4,3)
B3 = B2.expand(4,3)
C1 = B2 + A2
print(C1)

[[ 0]
 [10]
 [20]
 [30]] [0 1 2]
tensor([[ 0],
        [10],
        [20],
        [30]], dtype=torch.int32) tensor([0, 1, 2], dtype=torch.int32)
tensor([[ 0,  1,  2],
        [10, 11, 12],
        [20, 21, 22],
        [30, 31, 32]], dtype=torch.int32)
tensor([[0, 1, 2]], dtype=torch.int32)
tensor([[ 0,  1,  2],
        [10, 11, 12],
        [20, 21, 22],
        [30, 31, 32]], dtype=torch.int32)


### 逐元素操作
输入形状和输出形状相同

In [21]:
t = torch.randn(1, 3)
t1 = torch.randn(3, 1)
t2 = torch.randn(1, 3)
print(t, t1, t2)
# t + 0.1 * (t1/t2)
print(torch.addcdiv(t, 0.1, t2, t2))
# 计算sigmoid
print(torch.sigmoid(t))
# 将t限制在[0, 1]之间
print(torch.clamp(t, 0, 1))
print(t)
t.add_(2)
print(t)

tensor([[ 0.0751,  0.4777, -0.6759]]) tensor([[-2.1489],
        [-1.1463],
        [-0.2720]]) tensor([[ 1.0066, -0.0416, -1.2853]])
tensor([[ 0.1751,  0.5777, -0.5759]])
tensor([[0.5188, 0.6172, 0.3372]])
tensor([[0.0751, 0.4777, 0.0000]])
tensor([[ 0.0751,  0.4777, -0.6759]])
tensor([[2.0751, 2.4777, 1.3241]])


### 归并操作

In [23]:
a = torch.linspace(0, 10, 6)
print(a)
a = a.view((2, 3))
print(a)
b = a.sum(dim=0) # dim=0 表示沿着y轴进行相加
print(b)
b= a.sum(dim=0, keepdim=True)
print(b)

tensor([ 0.,  2.,  4.,  6.,  8., 10.])
tensor([[ 0.,  2.,  4.],
        [ 6.,  8., 10.]])
tensor([ 6., 10., 14.])
tensor([[ 6., 10., 14.]])


### 比较操作

In [25]:
import torch
x = torch.linspace(0 ,10, 6).view(2,3)
print(x)
print(torch.max(x))
print(torch.max(x, dim=0))
print(torch.topk(x, 2, dim=0))

tensor([[ 0.,  2.,  4.],
        [ 6.,  8., 10.]])
tensor(10.)
torch.return_types.max(
values=tensor([ 6.,  8., 10.]),
indices=tensor([1, 1, 1]))
torch.return_types.topk(
values=tensor([[ 6.,  8., 10.],
        [ 0.,  2.,  4.]]),
indices=tensor([[1, 1, 1],
        [0, 0, 0]]))


### 矩阵操作

In [27]:
a = torch.tensor([2,3])
b = torch.tensor([3,4])
print(torch.dot(a, b))
x = torch.randint(10, (2,3))
y = torch.randint(6, (3,4))
print(x, y)
t1 = torch.mm(x, y)
print(t1, t1.shape)
x = torch.randint(10, (2,4,3))
y = torch.randint(6, (2,3,4))
print(x, y)
print(torch.bmm(x, y))

tensor(18)
tensor([[0, 4, 8],
        [8, 6, 7]]) tensor([[2, 3, 1, 1],
        [1, 1, 5, 1],
        [3, 5, 4, 1]])
tensor([[28, 44, 52, 12],
        [43, 65, 66, 21]]) torch.Size([2, 4])
tensor([[[3, 0, 3],
         [9, 6, 9],
         [0, 7, 1],
         [7, 3, 8]],

        [[5, 9, 4],
         [6, 8, 2],
         [1, 7, 9],
         [0, 9, 0]]]) tensor([[[3, 4, 0, 3],
         [4, 5, 0, 3],
         [5, 5, 4, 1]],

        [[0, 0, 1, 0],
         [3, 0, 3, 0],
         [4, 5, 0, 2]]])
tensor([[[ 24,  27,  12,  12],
         [ 96, 111,  36,  54],
         [ 33,  40,   4,  22],
         [ 73,  83,  32,  38]],

        [[ 43,  20,  32,   8],
         [ 32,  10,  30,   4],
         [ 57,  45,  22,  18],
         [ 27,   0,  27,   0]]])


## 神经网络工具